In [0]:
import itertools as itt
from collections import Counter
from copy import deepcopy
from functools import partial
import torch
from tabulate import tabulate
import numpy as np 
import gym
import pyro
from gym.wrappers import TimeLimit
from pyro.distributions import Categorical, Delta, Normal
from pyro.infer import EmpiricalMarginal, Importance
from statistics import mean


# Causal Reinforcement learning
Reinforcement learning (RL) is the study of how an agent (human, animal or machine) can learn to choose actions that maximize its future rewards. The system consists of an agent that interacts  with the environment via actions at discrete time steps and recieves a certain reward. 
![alt text](https://i.imgur.com/EQmZWDc.png?1)

This problem is inherently causal in nature. We are interested in identifying the effect of taking an action and also the actions that give maximum rewards. This relates to the Bayesian Causal Inference where one can reason backwards to derive the causes based on the observation of effects. Applying this to the RL setting one can infer the best action based on the reward(Planning as inference). Can standard reinforcement learning derive this causal relationship between action and or is it merely an estimate of association.
Today we have state of the art Reinforcement learning algorithms that surpass humans in environments on which they are trained, but they lack the ability to learn reusable human-like concepts. Even minor changes to the environment leave them confused. 

A DAG for sequential decision making in reinforcement learning : 

![image](https://i.imgur.com/jS7vzSJ.png?1)

U here is a confounder that affects the action as well the outcomes(rewards) in the environment. Often these confounders are unobserved and in those situations the estimate of effect of an action on the outcome is biased. 





# Bellman Equation
The goal of an agent is to maximize the expected reward , which is defined as:

$$Q(s, a) = E[R|s_0=s, a_0=a]$$

The optimal action then for a given state s maximizes Q(s, a):

$$a^*_s = argmax_a Q(s, a)$$

Due to the markov property assumption we can express the optimal Q value as :
$$Q^*(s, a) = R(s, a) + \gamma \sum_{s'}T(s'|s,a) max_{a'} Q^*(s', a')$$ 

## Actions as Interventions 
In the causal world we can define the expected reward as :

$$Q(s, a) = E[R|s_0=s, do(a_0=a)]$$

The optimal value function :
$$Q^*(s, a) = R(s, a) + \gamma \sum_{s'}T(s'|s,a) max_{a'} [Q^*(s', a')| do(a = a')]$$ 

With actions as active interventions, we can now eliminate the confounding bias in the environment and arrive at a optimal policy.



# Experiments on FrozenLake-V0 from OpenAI Gym
Frozen Lake is a grid environment where some tiles are walkable and others lead the agent to fall into the water. Agent is rewarded for finding a walkable path to the goal. 

![alt text](https://i.imgur.com/g5gduIk.png)

A binary confounder was added to the environment. 
This confounder determines the action and reward distribution at each timestep. 




In [0]:

class FrozenLakeWrapper(gym.Wrapper):
    """FrozenLakeWrapper

    Wrapper around a FrozenLake environment; implements a time limit and reward
    shaping.
    Also includes a confounder
    :param env:  OpenAI Gym environment
    :param max_episode_steps:  problem horizon
    """

    def __init__(self, env, *, max_episode_steps):
        env = TimeLimit(env, max_episode_steps=max_episode_steps)
        super().__init__(env)
        self.actions = ['left', 'down', 'right', 'up']
        self.u = self.get_confounder()
        self.episodes = 0

    # generate a binary confounder 
    def get_confounder(self):
        return pyro.sample('u', Categorical(torch.tensor([1., 1.])))

    # get the reward based on the action and confounder
    def utility(self, action):
        r_vals = [0.2, 0.25]
        r_probs = [[[0.8, 0.2], [0.75, 0.25], [0.7, 0.3], [0.85, 0.15]], 
                [[0.6, 0.4], [0.65, 0.35], [0.7, 0.3], [0.8, 0.2]]]
        rc = pyro.sample(f'rc_{self.episodes}', Categorical(torch.tensor(r_probs[self.u][action])))
        self.episodes = self.episodes + 1
        return r_vals[rc.item()]
    
    # confounded action distribution
    def action_dist_confounded(self):
        action_probs = [[0.45, 0.45, 0.2, 0.2],[0.55, 0.55, 0.8, 0.8]]
        return Categorical(torch.tensor(action_probs[self.u]))


    # take a step in the environment
    def step(self, action):

        observation, reward, done, info = self.env.step(action)

        if info.get('TimeLimit.truncated', False):
            # negative reward if time limit is reached
            reward = -1.0
        elif done and reward == 0.0:
            # negative reward for falling in a hole
            reward = -1.0

        # reward has the original value
        # modify it to reflect the effect of confounding
        if reward == 0.:
            reward = reward + self.utility(action)

        return observation, reward , done, info


Causal model of the environment, confounded action and reward distributions

In [0]:
ACTIONS = [0, 1, 2, 3]
# causal model of the environment
def action_distribution(u=0):
    action_probs = torch.tensor([[0.25, 0.35, 0.2, 0.2],  # u = 0
                                 [0.75, 0.65, 0.8, 0.8]]) # u = 1
    return Categorical(action_probs[u])

def reward_distribution(state, action, u):
    # original reward distribution
    if state == 15.:
      reward = 1.
    elif state in [5, 7, 11, 12]:
      reward = -1.
    else:
      reward = 0. 
    # confounded distribution
    r_vals = [0.2, 0.4] # incremental reward value
    r_probs = [[[0.8, 0.2], [0.75, 0.25], [0.7, 0.3], [0.85, 0.15]], # u=0, action = a
                [[0.6, 0.4], [0.65, 0.35], [0.7, 0.3], [0.8, 0.2]]] # u = 1, action = a 
    rc = Categorical(torch.tensor(r_probs[int(u)][int(action)])).sample()
    if reward == 0. :
       reward += r_vals[int(rc)]
    # sample reward from Dirac
    return pyro.sample('reward', Normal(torch.tensor(reward), torch.tensor(0.00001)))


def model(env):
    """Model of the environment"""
    u = pyro.sample('u', Categorical(torch.tensor([1., 1.])))
    action = pyro.sample('action', action_distribution(u))
    observation, _, done, info = env.step(int(action))
    reward = reward_distribution(observation, action, u)
    return env, observation, reward, done, info

In [0]:
def argmax(iterable, func):
    """Get the argmax of an iterable"""
    return max(iterable, key=func)

def manhattan_distance(s, gridsize=4):
    """ Get the manhattan distance from goal"""
    # 2 dimensional vectors, s/4 gives row index , s%4 gives column index for a 4 * 4 grid
    p = np.array([int(s / gridsize), (s % gridsize)])
    q = np.array([gridsize, gridsize])
    return sum(abs(p - q))

def expected_reward(Q_function, action, env, u, i=0):
    def get_posterior_mean(posterior, n_samples=30):
        """
        Calculate posterior mean
        """
        # Sample
        marginal_dist = EmpiricalMarginal(posterior).sample((n_samples, 1)).float()
        # assumed to be all the same
        return torch.mean(marginal_dist)
    # The use of the param store is an optimization
    param_name = 'posterior_reward_state{}_{}'.format(env.s, i)
    if param_name in list(pyro.get_param_store().keys()):
        posterior_mean = pyro.get_param_store().get_param(param_name)
        return posterior_mean
    else:
        # this gets slower as we increase num_samples
        inference = Importance(Q_function, num_samples=30)
        posterior = inference.run(action, env, u)
        posterior_mean = get_posterior_mean(posterior, 30)
        pyro.param(param_name, posterior_mean)
        return posterior_mean


def imagine_next_step(env, action):
    """Agent imagines next time step"""
    sim_env = deepcopy(env)
    state = sim_env.s
    # when I intervene I should also make sure u does not change over time in this environment
    int_model = pyro.do(model, {'action': action})
    sim_env, _, _, _, _ = int_model(sim_env)
    # sanity check
    assert sim_env.lastaction == action
    return sim_env


def Q(action, env, u=0):
    """Q function variant of Bellman equation."""
    utility = reward_distribution(env.s, action, u)
    if utility not in [1., -1]:
        env_step = imagine_next_step(env, action)
    # check if the action got us closer to the goal. if yes only then recurse
        if (env.s != env_step.s) and (manhattan_distance(env.s) >= manhattan_distance(env_step.s)) :
            # Calculate expected rewards for each action but
            # exclude backtracking actions.
            expected_rewards = [Q(act, env_step, u)
                for j, act in enumerate(ACTIONS)
                if ACTIONS[abs(j - 2)] != action
            ]
            # Choose reward from optimal action
            utility = utility + max(expected_rewards)
    return utility



def policy(real_env, u,i=0):
    # Choose optimal action
    action = argmax(ACTIONS, partial(Q, env=real_env, u=u ))
    print(action)
    return action


def main(fl_env):
    """A walk through the environment by choosing optimal actions"""
    u = fl_env.u
    for t in range(100):
        pyro.clear_param_store()
        action = policy(fl_env, u) 
        int_model = pyro.do(model,
                            {'action': torch.tensor(action)})
        fl_env, observation, reward, done, info = int_model(fl_env)
        fl_env.render()
        if done:
            print("Episode finished after {} timesteps".format(t + 1))
            break
    fl_env.close()
    


In [0]:
env = gym.make('FrozenLake-v0', is_slippery=False)
fl_env = FrozenLakeWrapper(env, max_episode_steps=50)

The agent is able to finally the reach the goal but it may not have made the optimal choice at each step. If it did, it should reach the goal in 6 steps 

In [125]:
# render the results
fl_env.reset()
main(fl_env)

2
  (Right)
SFFF
FHFH
FFFH
HFFG
2
  (Right)
SFFF
FHFH
FFFH
HFFG
2
  (Right)
SFFF
FHFH
FFFH
HFFG
3
  (Up)
SFFF
FHFH
FFFH
HFFG
3
  (Up)
SFFF
FHFH
FFFH
HFFG
0
  (Left)
SFFF
FHFH
FFFH
HFFG
2
  (Right)
SFFF
FHFH
FFFH
HFFG
0
  (Left)
SFFF
FHFH
FFFH
HFFG
1
  (Down)
SFFF
FHFH
FFFH
HFFG
1
  (Down)
SFFF
FHFH
FFFH
HFFG
1
  (Down)
SFFF
FHFH
FFFH
HFFG
2
  (Right)
SFFF
FHFH
FFFH
HFFG
Episode finished after 12 timesteps
